## Ordinal Classification
### Model Serving

In [1]:
import pandas as pd

# Features verification
from feature_engine.preprocessing import MatchVariables
# Categories verification
from feature_engine.preprocessing import MatchCategories
# Script Serving
from jpmml_evaluator import LoadingModelEvaluatorBuilder
from jpmml_evaluator.py4j import Py4JBackend
# REST Serving
from openscoring import Openscoring

In [29]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Data Import
df = pd.read_csv('../../../data/winequality-white-cleaned.csv',sep=';')
# Get sample dataset for model serving
df_serve = df.sample(30)
x_serve = df_serve.drop('target', axis=1)
y_serve = df_serve[['target']]
x_serve.reset_index(drop=True, inplace=True)
y_serve.reset_index(drop=True, inplace=True)

## Features Verification

In [3]:
# Ensures that columns in serving set match those in train set
x_train = df_serve.drop('target', axis=1)
# Setup the transformer
match_cols = MatchVariables(missing_values="ignore")
match_cols.fit(x_train)
match_cols.feature_names_in_

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [4]:
# Run the transformer
x_match = match_cols.transform(x_serve)
x_match.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,6.510377,0.164937,0.314025,5.432389,0.031954,58.253143,127.701257,0.991603,3.225849,0.701038,12.219497
1,6.025079,0.167492,0.332508,6.025079,0.035498,30.250788,111.752364,0.993628,3.320000,0.580000,10.150000
2,7.033988,0.268079,0.371921,2.221178,0.045596,34.000000,106.403926,0.990491,3.237634,0.508719,12.800000
3,5.619242,0.294196,0.256593,2.652678,0.072502,49.451101,165.709780,0.994943,3.281136,0.528391,8.729022
4,6.511633,0.425117,0.327210,4.106998,0.032721,48.093002,163.697667,0.992730,3.433254,0.656974,11.851166


## Categories Verification

In [5]:
# Ensures categorical variables are of type ‘category’
mc = MatchCategories(missing_values="ignore")
# mc.fit(x_train)

In [6]:
# Run the transformer
# x_match_c = mc.transform(x_match)
# x_match_c.head(5)

## Model Script Serving

In [7]:
# Load PMML model
py4jbackend  = Py4JBackend()
evaluator = LoadingModelEvaluatorBuilder(backend = py4jbackend ) \
    .loadFile("../../../data/OrdinalLogisticRegression.pmml") \
    .build()

In [8]:
# Perform automated QA
evaluator.verify()

In [9]:
# Model Serving
dfresult = evaluator.evaluateAll(x_match)
dfresult.head(5)

,target,probability(3),probability(4),probability(5),probability(6),probability(7),probability(8),probability(9)
0,8,0.000564,0.000190,0.000144,0.007457,0.340096,0.648422,3.126919e-03
1,7,0.033063,0.059658,0.108570,0.158700,0.362257,0.277750,2.270307e-06
2,8,0.017782,0.006642,0.000632,0.006408,0.269123,0.699265,1.474899e-04
3,4,0.168523,0.486868,0.320300,0.016670,0.004930,0.002709,4.163612e-20
4,8,0.099366,0.073194,0.008050,0.009112,0.298173,0.507774,4.331617e-03


## Model REST Serving

In [10]:
# Creating an Openscoring object
# Openscoring server have to be running on base-url
os = Openscoring(base_url = "http://localhost:8080", token="secret")

In [11]:
# Deploying a PMML model:
model_name = 'OrdinalLogisticRegression'
os.deployFile(model_name, "../../../data/OrdinalLogisticRegression.pmml")

In [12]:
# Model serving with data records from DataFrame
dfResponse = os.evaluateCsv(model_name, x_match)
print(dfResponse.head(5))

   target  probability(3)  probability(4)  probability(5)  probability(6)  \
0       8        0.000564        0.000190        0.000144        0.007457   
1       7        0.033063        0.059658        0.108570        0.158700   
2       8        0.017782        0.006642        0.000632        0.006408   
3       4        0.168523        0.486868        0.320300        0.016670   
4       8        0.099366        0.073194        0.008050        0.009112   

   probability(7)  probability(8)  probability(9)  
0        0.340096        0.648422    3.126919e-03  
1        0.362257        0.277750    2.270307e-06  
2        0.269123        0.699265    1.474899e-04  
3        0.004930        0.002709    4.163612e-20  
4        0.298173        0.507774    4.331617e-03  
